In [1]:
# construct the matrices for all possible models
# there are 2^16 of these
# here is one easy (silly?) way to do it

rows = []

for i in [0,1]:
    for j in [0,1]:
        for k in [0,1]:
            for l in [0,1]:
                rows.append([i,j,k,l])

allmodels = []

for r1 in rows:
    for r2 in rows:
        for r3 in rows:
            for r4 in rows:
                allmodels.append(matrix([r1,r2,r3,r4]))

len(allmodels)               

65536

In [2]:
# now we want to only keep those models which are connected
# ie. those for which there is a power n>1 such that (T^n)_ij > 0
# equivalently, is there an n>1 such that T+T^2+...+T^n has no zeros?

def is_connected(T,n):
    S = zero_matrix(4)
    for i in range(1,n+1):
        S += T^i
    return(min(S.list()) > 0)

connectedmodels = []

# n=4 suffices
for T in allmodels:
    if is_connected(T, 4):
        connectedmodels.append(T)

len(connectedmodels)

25696

In [3]:
# now we want the aperiodic models
# that is, models for which there is some n such that T^n has no zeros

def is_aperiodic(T,n):
    S = T^n
    return(min(S.list()) > 0)

aperiodicmodels = []

# it can be shown that n=10 is sufficient to check
for T in connectedmodels:
    if is_aperiodic(T, 10):
        aperiodicmodels.append(T)

len(aperiodicmodels)

25575

In [4]:
# want to find all the non-isomorphic models in the full plane
# any symmetries are allowed, ie. all of S4
# we can calculate the number quickly using Burnside's lemma

S4 = Permutations(4)
S4M = [P.to_matrix() for P in S4]

fixedall = [0 for i in range(24)]

for i in range(24):
    print(i)
    for T in allmodels:
        if S4M[i].transpose()*T*S4M[i] == T:
            fixedall[i] += 1

numfixedall = sum(fixedall)/24
print('\n', 'numfixedall', numfixedall, '\n')

fixedconnected = [0 for i in range(24)]

for i in range(24):
    print(i)
    for T in connectedmodels:
        if S4M[i].transpose()*T*S4M[i] == T:
            fixedconnected[i] += 1

numfixedconnected = sum(fixedconnected)/24
print('\n', 'numfixedconnected', numfixedconnected, '\n')

fixedaperiodic = [0 for i in range(24)]

for i in range(24):
    print(i)
    for T in aperiodicmodels:
        if S4M[i].transpose()*T*S4M[i] == T:
            fixedaperiodic[i] += 1

numfixedaperiodic = sum(fixedaperiodic)/24
print('\n', 'numfixedaperiodic', numfixedaperiodic)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23

 numfixedall 3044 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23

 numfixedconnected 1168 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23

 numfixedaperiodic 1159


In [36]:
# actually computing a representative from each isomorphism class takes a bit more work

aperiodicnonisomodels = []

aperiodicmodelscopy = aperiodicmodels.copy()

i = 0
while i < len(aperiodicmodelscopy):
    if i%100 == 0:
        print(i, len(aperiodicmodelscopy))
    T = aperiodicmodelscopy[i]
    aperiodicnonisomodels.append(T)
    for S in S4M:
        if S.transpose()*T*S in aperiodicmodelscopy[i+1:]:
            aperiodicmodelscopy.remove(S.transpose()*T*S)
    i += 1

print('\n', len(aperiodicnonisomodels))

0 25575
100 23607
200 21331
300 19031
400 16755
500 14551
600 12443
700 10437
800 8259
900 6151
1000 4055
1100 2264

 1159 



In [27]:
# print the aperiodic matrices to file
    
aperiodicmodelsstr = []

for T in aperiodicnonisomodels:
    Ts = ''
    for row in T:
        for i in row:
            Ts += str(i)
    aperiodicmodelsstr.append(Ts)
    
with open('full_plane_models.txt', 'w') as f:
    for Ts in aperiodicmodelsstr:
        f.write("%s\n" % Ts)

In [54]:
# next we go to the half plane
# we are only interested in connected models
# the interesting models here are the vertically unbounded ones
# ie. those which are neither north-bound (must be a north step between two south steps) nor south-bound (must be a south step between two north steps)

def is_northbound(T, n):
    TnoN = diagonal_matrix([1,0,1,1])*T*diagonal_matrix([1,0,1,1])
    S = zero_matrix(4)
    for i in range(1,n+1):
        S += TnoN^i
    return(S[3,3] == 0)

def is_southbound(T, n):
    TnoS = diagonal_matrix([1,1,1,0])*T*diagonal_matrix([1,1,1,0])
    S = zero_matrix(4)
    for i in range(1,n+1):
        S += TnoS^i
    return(S[1,1] == 0)

vunbconnectedmodels = []

# n=3 suffices
for T in connectedmodels:
    if not is_northbound(T, 3) and not is_southbound(T, 3):
        vunbconnectedmodels.append(T)
        
print(len(vunbconnectedmodels))

vunbaperiodicmodels = []

for T in aperiodicmodels:
    if not is_northbound(T, 3) and not is_southbound(T, 3):
        vunbaperiodicmodels.append(T)
        
print(len(vunbaperiodicmodels))

19328
19285


In [57]:
# now do the Burnside calculations for the half plane
# here instead of S4 the only symmetry we care about is x <-> -x

# the relevant permutation matrix is
SH = matrix([[0,0,1,0],[0,1,0,0],[1,0,0,0],[0,0,0,1]])

halffixedconnected = 0

for T in vunbconnectedmodels:
    if SH.transpose()*T*SH == T:
        halffixedconnected += 1

numhalffixedconnected = (len(vunbconnectedmodels) + halffixedconnected)/2
print('numhalffixedconnected', numhalffixedconnected)

halffixedaperiodic = 0

for T in vunbaperiodicmodels:
    if SH.transpose()*T*SH == T:
        halffixedaperiodic += 1

# the identity fixes everything        
numhalffixedaperiodic = (len(vunbaperiodicmodels) + halffixedaperiodic)/2
print('numhalffixedaperiodic', numhalffixedaperiodic)

numhalffixedconnected 9744
numhalffixedaperiodic 9722


In [59]:
# compute a representative from each isomorphism class

aperiodichalfmodels = []

vunbaperiodicmodelscopy = vunbaperiodicmodels.copy()

i = 0
while i < len(vunbaperiodicmodelscopy):
    if i%1000 == 0:
        print(i, len(vunbaperiodicmodelscopy))
    T = vunbaperiodicmodelscopy[i]
    aperiodichalfmodels.append(T)
    if SH.transpose()*T*SH in vunbaperiodicmodelscopy[i+1:]:
        vunbaperiodicmodelscopy.remove(SH.transpose()*T*SH)
    i += 1

print('\n', len(aperiodichalfmodels))

0 19285
1000 18293
2000 17293
3000 16295
4000 15309
5000 14327
6000 13337
7000 12354
8000 11372
9000 10385

 9722 



In [60]:
# print to file
    
aperiodichalfmodelsstr = []

for T in aperiodichalfmodels:
    Ts = ''
    for row in T:
        for i in row:
            Ts += str(i)
    aperiodichalfmodelsstr.append(Ts)
    
with open('half_plane_models.txt', 'w') as f:
    for Ts in aperiodichalfmodelsstr:
        f.write("%s\n" % Ts)

In [63]:
# finally to the quarter plane
# there are a bunch of conditions to consider here

# first we want models which are horizontally unbounded as well

def is_eastbound(T, n):
    TnoE = diagonal_matrix([0,1,1,1])*T*diagonal_matrix([0,1,1,1])
    S = zero_matrix(4)
    for i in range(1,n+1):
        S += TnoE^i
    return(S[2,2] == 0)

def is_westbound(T, n):
    TnoW = diagonal_matrix([1,1,0,1])*T*diagonal_matrix([1,1,0,1])
    S = zero_matrix(4)
    for i in range(1,n+1):
        S += TnoW^i
    return(S[0,0] == 0)

cunbconnectedmodels = []

# n=3 suffices
for T in vunbconnectedmodels:
    if not is_westbound(T, 3) and not is_eastbound(T, 3):
        cunbconnectedmodels.append(T)
        
print(len(cunbconnectedmodels))

cunbaperiodicmodels = []

for T in vunbaperiodicmodels:
    if not is_westbound(T, 3) and not is_eastbound(T, 3):
        cunbaperiodicmodels.append(T)
        
print(len(cunbaperiodicmodels))

14978
14943


In [66]:
# we can do Burnside's lemma on those two sets
# the only symmetry is x <-> y

# the matrix is
SQ = matrix([[0,1,0,0],[1,0,0,0],[0,0,0,1],[0,0,1,0]])

quarterfixedconnected = 0

for T in cunbconnectedmodels:
    if SQ.transpose()*T*SQ == T:
        quarterfixedconnected += 1

numquarterfixedconnected = (len(cunbconnectedmodels) + quarterfixedconnected)/2
print('numquarterfixedconnected', numquarterfixedconnected)

quarterfixedaperiodic = 0

for T in cunbaperiodicmodels:
    if SQ.transpose()*T*SQ == T:
        quarterfixedaperiodic += 1

# the identity fixes everything        
numquarterfixedaperiodic = (len(cunbaperiodicmodels) + quarterfixedaperiodic)/2
print('numquarterfixedaperiodic', numquarterfixedaperiodic)

numquarterfixedconnected 7541
numquarterfixedaperiodic 7520


In [64]:
# next we need them to be diagonally unbounded
# that is, we want walks to be able to go arbitrarily far above the line y=-x, and arbitrarily far above and below the line y=x
# the conditions to check this are the following

def is_southwestbound(T):
    return(T[0,0] == 0 and T[1,1] == 0 and T[1,0]*T[0,1] == 0 and T[0,1]*T[1,3]*T[3,0] == 0 and T[0,3]*T[3,1]*T[1,0] == 0 and T[1,0]*T[0,2]*T[2,1] == 0 and T[1,2]*T[2,0]*T[0,1] == 0)

def is_southeastbound(T):
    return(T[1,1] == 0 and T[2,2] == 0 and T[1,2]*T[2,1] == 0 and T[1,0]*T[0,2]*T[2,1] == 0 and T[1,2]*T[2,0]*T[0,1] == 0 and T[2,1]*T[1,3]*T[3,2] == 0 and T[2,3]*T[3,1]*T[1,2] == 0)

def is_northwestbound(T):
    return(T[0,0] == 0 and T[3,3] == 0 and T[0,3]*T[3,0] == 0 and T[0,1]*T[1,3]*T[3,0] == 0 and T[0,3]*T[3,1]*T[1,0] == 0 and T[3,0]*T[0,2]*T[2,3] == 0 and T[3,2]*T[2,0]*T[0,3] == 0)

cdunbconnectedmodels = []

for T in cunbconnectedmodels:
    if not is_southwestbound(T) and not is_southeastbound(T) and not is_northwestbound(T):
        cdunbconnectedmodels.append(T)
        
print(len(cdunbconnectedmodels))

cdunbaperiodicmodels = []

for T in cunbaperiodicmodels:
    if not is_southwestbound(T) and not is_southeastbound(T) and not is_northwestbound(T):
        cdunbaperiodicmodels.append(T)
        
print(len(cdunbaperiodicmodels))

14209
14205


In [67]:
# Burnside again

quarterfixedconnectedv2 = 0

for T in cdunbconnectedmodels:
    if SQ.transpose()*T*SQ == T:
        quarterfixedconnectedv2 += 1

numquarterfixedconnectedv2 = (len(cdunbconnectedmodels) + quarterfixedconnectedv2)/2
print('numquarterfixedconnectedv2', numquarterfixedconnectedv2)

quarterfixedaperiodicv2 = 0

for T in cdunbaperiodicmodels:
    if SQ.transpose()*T*SQ == T:
        quarterfixedaperiodicv2 += 1

# the identity fixes everything        
numquarterfixedaperiodicv2 = (len(cdunbaperiodicmodels) + quarterfixedaperiodicv2)/2
print('numquarterfixedaperiodicv2', numquarterfixedaperiodicv2)

numquarterfixedconnectedv2 7149
numquarterfixedaperiodicv2 7146


In [68]:
# the last condition to check is whether the model is glued
# that is, if restricted to the quarter plane, will it actually be able to leave the x and y axes?

# the condition is this

def is_glued(T):
    return(T[0,1] == 0 and T[0,0]*T[0,2]*T[2,1] == 0 and T[1,0] == 0 and T[1,1]*T[1,3]*T[3,0] == 0)

ngcdunbconnectedmodels = []

for T in cdunbconnectedmodels:
    if not is_glued(T):
        ngcdunbconnectedmodels.append(T)

print(len(ngcdunbconnectedmodels))

ngcdunbaperiodicmodels = []

for T in cdunbaperiodicmodels:
    if not is_glued(T):
        ngcdunbaperiodicmodels.append(T)

print(len(ngcdunbaperiodicmodels))

13749
13745


In [69]:
# a final Burnside

quarterfixedconnectedv3 = 0

for T in ngcdunbconnectedmodels:
    if SQ.transpose()*T*SQ == T:
        quarterfixedconnectedv3 += 1

numquarterfixedconnectedv3 = (len(ngcdunbconnectedmodels) + quarterfixedconnectedv3)/2
print('numquarterfixedconnectedv3', numquarterfixedconnectedv3)

quarterfixedaperiodicv3 = 0

for T in ngcdunbaperiodicmodels:
    if SQ.transpose()*T*SQ == T:
        quarterfixedaperiodicv3 += 1

# the identity fixes everything        
numquarterfixedaperiodicv3 = (len(ngcdunbaperiodicmodels) + quarterfixedaperiodicv3)/2
print('numquarterfixedaperiodicv3', numquarterfixedaperiodicv3)

numquarterfixedconnectedv3 6912
numquarterfixedaperiodicv3 6909


In [70]:
# compute a representative from each isomorphism class

aperiodicquartermodels = []

ngcdunbaperiodicmodelscopy = ngcdunbaperiodicmodels.copy()

i = 0
while i < len(ngcdunbaperiodicmodelscopy):
    if i%1000 == 0:
        print(i, len(ngcdunbaperiodicmodelscopy))
    T = ngcdunbaperiodicmodelscopy[i]
    aperiodicquartermodels.append(T)
    if SQ.transpose()*T*SQ in ngcdunbaperiodicmodelscopy[i+1:]:
        ngcdunbaperiodicmodelscopy.remove(SQ.transpose()*T*SQ)
    i += 1

print('\n', len(aperiodicquartermodels))

0 13745
1000 12745
2000 11745
3000 10754
4000 9773
5000 8773
6000 7787

 6909 



In [71]:
# and print to file

aperiodicquartermodelsstr = []

for T in aperiodicquartermodels:
    Ts = ''
    for row in T:
        for i in row:
            Ts += str(i)
    aperiodicquartermodelsstr.append(Ts)
    
with open('quarter_plane_models.txt', 'w') as f:
    for Ts in aperiodicquartermodelsstr:
        f.write("%s\n" % Ts)